In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

In [3]:
with open('../input/nerenglishlarge/Date.pkl', 'rb') as f:
    Date = pickle.load(f)
with open('../input/nerenglishlarge/Datebeforefilter.pkl', 'rb') as f:
    date_before_Filter = pickle.load(f)
with open('../input/nerenglishlarge/LOClarge.pkl', 'rb') as f:
    Loclarge = pickle.load(f)
with open('../input/nerenglishlarge/ORGlarge.pkl', 'rb') as f:
    Orglarge = pickle.load(f)
with open('../input/nerenglishlarge/Personlarge.pkl', 'rb') as f:
    Personlarge = pickle.load(f)
with open('../input/nerenglishlarge/Persononto.pkl', 'rb') as f:
    Person_onto = pickle.load(f)
with open('../input/nerenglishlarge/finaldate.pkl', 'rb') as f:
    date_after_fltr = pickle.load(f)
with open('../input/final-d/Final_date.pkl', 'rb') as f:
    final_date = pickle.load(f)
with open('../input/nerenglishlarge/nerengentity.pkl', 'rb') as f:
    Ner_eng = pickle.load(f)
with open('../input/nerenglishlarge/nerenglargegentity.pkl', 'rb') as f:
    Ner_eng_large = pickle.load(f)
with open('../input/nerenglishlarge/ontoentity.pkl', 'rb') as f:
    ontonotes = pickle.load(f)

In [4]:
data=pd.read_csv("../input/records/records.csv")
all_text=data["transcription_text"]

In [5]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s
start=time.time()
for i in tqdm(range(10000)):
    all_text[i] = process(all_text[i])
for i in tqdm(range(10000)):
    all_text[i] = " ".join(all_text[i])
end=time.time()
print(end-start)

100%|██████████| 10000/10000 [00:06<00:00, 1617.74it/s]

38.38823127746582


In [6]:
Date[:10]

[['March 19th 1866', 'March 19 1866'],
 ['Mon 28 . 1866'],
 ['twenty one', 'twenty one year old'],
 [],
 [],
 ['twelve month', 'five year'],
 [],
 [],
 ['Sunday', 'early last spring', 'Christmas'],
 []]

In [7]:
all_text[500]

'10 Complaint Disposition Nov 4 1867 Mr. R. C. Stuart , White , complains one Jacob Corcoran . contracting work one year , left Plantation gathering crop c. Letter sent Corcoran appear office Thursday Nov 7 , 1867 explain reason leaving Mr. Stuart crop gathered . Jake appeared case settled . Nov 5 1867 Mr. Z. L. Morris complains Zenas Hunter Dave Wallace laboring according contract Nov 5th 1867 Letter sent Zenas Hunter Dave Wallace appear office Thursday Nov 7 1867 bring account c Dave appeared contract explained . Nov 6 1867 E. Lowengard , white , complains one Isaac Harry owes 240.00 doll . pay anything . Said Isaac owes Mr. Lowengard good c purchased Isaac paid Mr. Lowengard 100.00 doll . Nov 7 1867 Willoby Sawyer complains one Windyke owes 20 bushel of corn 3 12 month labor February 24 , 1867 June 67 . Refuses pay anything c. Letter sent Windyke telling settle Sawyer appear office Thursday Nov 14 1867 . Both party appeared office case settled . Vandike giving Willoughby 9 Bu of cor

In [8]:
PL=[]
empty=0
for i in Date:
    for j in i:
        PL.append(j)
for i in Date:
    if i==[]:
        empty=empty+1
len(PL), empty

(22658, 1921)

In [9]:
PL=[]
empty=0
for i in date_before_Filter:
    for j in i:
        PL.append(j)
for i in date_before_Filter:
    if i==[]:
        empty=empty+1
len(PL), empty

(12186, 3573)

In [10]:
PL=[]
empty=0
for i in Personlarge:
    for j in i:
        PL.append(j)
for i in Personlarge:
    if i==[]:
        empty=empty+1
len(PL), empty

(125981, 7254)

In [11]:
PL=[]
empty=0
for i in Person_onto:
    for j in i:
        PL.append(j)
for i in Person_onto:
    if i==[]:
        empty=empty+1
len(PL), empty

(37349, 1966)

In [12]:
PL=[]
empty=0
for i in Loclarge:
    for j in i:
        PL.append(j)
for i in Loclarge:
    if i==[]:
        empty=empty+1
len(PL), empty

(83375, 9965)

In [13]:
PL=[]
empty=0
for i in Orglarge:
    for j in i:
        PL.append(j)
for i in Orglarge:
    if i==[]:
        empty=empty+1
len(PL), empty

(54834, 14308)

In [14]:
PL=[]
empty=0
for i in final_date:
    for j in i:
        PL.append(j)
for i in final_date:
    if i==[]:
        empty=empty+1
len(PL), empty

(9193, 5036)

In [15]:
final_date

[['19-March-1866', '19-March-1866'],
 ['28-Mon-1866'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['17-July-1865'],
 [],
 [],
 ['17-July-1865'],
 [],
 [],
 [],
 ['27-July-1865'],
 [],
 [],
 [],
 [],
 ['30-July-1865'],
 ['24-July-1865', '29-June-1865', '27-June-1865'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['27-April-1865'],
 [],
 [],
 ['21-August-1865'],
 [],
 [],
 [],
 [],
 ['Month-August-1865'],
 [],
 ['10-Oct.-1865'],
 ['Month-Septembre-1865'],
 ['30-Septembre-1865'],
 [],
 ['Month-October-1865'],
 ['31-October-1865', 'Month-August-1865'],
 ['Month-August-1865'],
 ['17-October-1865'],
 [],
 [],
 [],
 ['16-October-1865', '16-October-1865'],
 [],
 [],
 ['20-October-1865', '27-October-1865', '23-October-1865'],
 [],
 ['14-Dec.-1865'],
 [],
 [],
 ['16-November-1865', '13-November-1865', '13-November-1865'],
 [],
 [],
 [],
 ['16-Decr-1865'],
 [],
 [],
 [],
 ['10-August-1865', '11-December-1865'],
 [],
 ['11-